In [1]:
import pandas as pd
import numpy as np
from scipy.stats import gaussian_kde
import os

In [2]:
del_dir1='genus_normalization_test'
del_dir2='Bimodal genus'
os.makedirs(del_dir2,exist_ok=True) 
filelist=os.listdir(del_dir1)
for file in filelist:
    del_file=del_dir1+'/'+file
    dT=pd.read_csv(del_file,index_col=0,delimiter='\t') 
    #1.Finding Biomodal Distribution of genes
    Bimodal_genusnames=[]
    Bimodal_genusvalleys=[]
    for i in range(len(dT.columns)):
        y=dT.iloc[:,i]
        if np.sum(abs(np.log(y[y>0]))>0)>0.3*len(dT): 
            data = np.log(y[y>0]) 
            #sns.kdeplot(data,bw_adjust=1) 
            kde = gaussian_kde(data) 
            kde_data = kde.evaluate(np.linspace(min(data), max(data), 1000))  
            kde_x=np.linspace(min(data), max(data), 1000) 
            kde_y=kde(kde_x) 
            dy_dx = np.gradient(kde_y, kde_x) 
            peakposi_y=[] 
            valeyposi_y=[] 
            peakposi_x=[] 
            valeyposi_x=[] 
            for j in range(len(dy_dx)):
                if dy_dx[j-1]>=0 and dy_dx[j]<=0 :  
                    peak_y=kde_y[j-1]
                    peakposi_y.append(peak_y)
                    peak_x=kde_x[j-1]
                    peakposi_x.append(peak_x)
                if dy_dx[j-1]<=0 and dy_dx[j]>=0:
                    valey_y=kde_y[j]
                    valeyposi_y.append(valey_y)
                    valey_x=kde_x[j]
                    valeyposi_x.append(valey_x)
            if len(peakposi_y)>=2: 
                Ymax1=max(peakposi_y) 
                BB=[]
                for a in peakposi_y: 
                    BB.append(a)
                BB.remove(max(BB))
                Ymax2=max(BB) 
                Xmax1=peakposi_x[peakposi_y.index(Ymax1)] 
                Xmax2=peakposi_x[peakposi_y.index(Ymax2)] 
                Yvaley=[] 
                Xvaley=[] 
                for b in valeyposi_x:
                    if Xmax1<b<=Xmax2 or Xmax2<b<=Xmax1:
                        CC=valeyposi_y[valeyposi_x.index(b)]
                        DD=b
                        Yvaley.append(CC)
                        Xvaley.append(DD)
                if Ymax2>=0.3*Ymax1 and (Ymax2-min(Yvaley))>=0.2*Ymax2 : 
                    posi=i
                    Bimodal_genusname=dT.columns[i] 
                    Bimodal_genusnames.append(Bimodal_genusname)
                    Bimodal_genusvalley=Xvaley[Yvaley.index(min(Yvaley))]
                    Bimodal_genusvalleys.append(Bimodal_genusvalley)
    pd.DataFrame({'Bimodal_genus_names':pd.Series(Bimodal_genusnames),'Bimodal_genus_valleys':pd.Series(Bimodal_genusvalleys)}).to_csv(del_dir2+'/'+file+'_BiDgenus',index=0,sep='\t') #把list转换为DataFrame后，保存为csv文件。
    print({file:len(Bimodal_genusnames)})

{'3_countries_combined_genus_normalization': 7}
{'American_genus_normalization': 4}
{'Danish_genus_normalization': 7}
{'Spanish_genus_normalization': 6}
